<a href="https://colab.research.google.com/github/d-m-silva/30DaysOfFLCode/blob/main/Dist_based_label_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Xtra-Computing/NIID-Bench

Cloning into 'NIID-Bench'...
remote: Enumerating objects: 485, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 485 (delta 198), reused 143 (delta 143), pack-reused 267 (from 1)
Receiving objects: 100% (485/485), 1.13 MiB | 21.40 MiB/s, done.
Resolving deltas: 100% (288/288), done.


In [ ]:
cd NIID-Bench/

**Distribution-based label imbalance**

-> Cifar-10, 10 parties, sample rate = 1, batch size = 64, learning rate = 0.01

In [ ]:
!python experiments.py --model=simple-cnn --dataset=cifar10 --alg=fedprox --lr=0.01 --batch-size=64 --epochs=5 --n_parties=10 --mu=0.01 --rho=0.9 --comm_round=50 --partition=noniid-labeldir --beta=0.5 --device='cpu' --datadir='./data/' --logdir='./logs/' --noise=0 --sample=1 --init_seed=0
!python experiments.py --model=simple-cnn --dataset=cifar10 --alg=fedavg --lr=0.01 --batch-size=64 --epochs=5 --n_parties=10 --mu=0.01 --rho=0.9 --comm_round=50 --partition=noniid-labeldir --beta=0.5 --device='cpu' --datadir='./data/' --logdir='./logs/' --noise=0 --sample=1 --init_seed=0
!python experiments.py --model=simple-cnn --dataset=cifar10 --alg=scaffold --lr=0.01 --batch-size=64 --epochs=5 --n_parties=10 --mu=0.01 --rho=0.9 --comm_round=50 --partition=noniid-labeldir --beta=0.5 --device='cpu' --datadir='./data/' --logdir='./logs/' --noise=0 --sample=1 --init_seed=0
!python experiments.py --model=simple-cnn --dataset=cifar10 --alg=fednova --lr=0.01 --batch-size=64 --epochs=5 --n_parties=10 --mu=0.01 --rho=0.9 --comm_round=50 --partition=noniid-labeldir --beta=0.5 --device='cpu' --datadir='./data/' --logdir='./logs/' --noise=0 --sample=1 --init_seed=0


In [ ]:
import glob
import json
import ast


def string_to_dict(string):
    """
    Converts a string representing a Namespace object to a dictionary.

    Args:
        string: The string to be converted.

    Returns:
        A dictionary containing the key-value pairs from the Namespace.
    """
    result = {}
    pairs = string.split(',')
    for pair in pairs:
        key, value = pair.split('=')
        key = key.strip()
        value = value.strip().strip("'")
        result[key] = value

    # Clean keys
    cleaned_keys = [key[11:].strip('"') if key.startswith('"Namespace(') else key.strip('"') for key in result.keys()]

    # Create a new dictionary with cleaned keys
    cleaned_result = {cleaned_keys[i]: result[list(result.keys())[i]] for i in range(len(cleaned_keys))}

    return cleaned_result



# Find most recent file (json)
json_files = glob.glob('logs/experiment_arguments*')
if json_files:
    latest_json = max(json_files, key=lambda x: x.split('-')[2])  # Order by timestamp

    # Open and extract important parameters from json file
    with open(latest_json, 'r') as json_file:

        content = json_file.read()
        args = string_to_dict(content)


        # Extract important info
        partition = args.get('partition', None)
        beta = args.get('beta', None)
        dataset = args.get('dataset', None)
        alg = args.get('alg', None)
        model = args.get('model', None)

else:
    print("No file found")
    partition = beta = dataset = alg = model = None


# Find most recent file (log)
log_files = glob.glob('logs/experiment_log*')
if log_files:
    latest_log = max(log_files, key=lambda x: x.split('-')[2])  #Order by timestamp

    last_test_accuracy = None

    with open(latest_log, 'r') as file:
        for line in file:
            if "Global Model Test accuracy" in line:
                last_test_accuracy = line.strip()
                last_test_accuracy = float(last_test_accuracy.rsplit(':', 1)[-1].strip())
                break  #To guarantee we get only the last ocurrence

else:
    print("No file found.")
    last_test_accuracy = None



# Creating dictionary 'results'
if all([partition, beta, dataset, alg, model, last_test_accuracy]):
    results = {
        'model': model,
        'dataset': dataset,
        'alg': alg,
        'beta': beta,
        'partition': partition,
        'last_test_accuracy': last_test_accuracy
    }
    print(results)
else:
    print("Parameters missing!")

